Mögliche Labels:
- eindeutige Positionierung der Zielspalte
- mehrfache Positionierungen der Zielspalten sortiert nach derer Kosinus-Ähnlichkeit

In [1]:
from utils.preprocessing import WordEmbedding, load_word_emb
import random
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import json
import numpy as np
from tqdm import tqdm

In [2]:
w2v_config = {
    'data_dir': 'data/glove',
    'word2idx_path': 'word2idx.json',
    'usedwordemb_path': 'usedwordemb.npy'
}
w2v = WordEmbedding(load_word_emb(w2v_config['data_dir'], 
                                  w2v_config['word2idx_path'],
                                  w2v_config['usedwordemb_path']))

In [3]:
# load table schemata
with open('data/wikisql/tables.jsonl') as file:
    table_schemata = pd.DataFrame([json.loads(line) for line in file.readlines()])
table_schemata['fits'] = ''
table_schemata['header'] = table_schemata['header'].apply(lambda x: '<|>'.join(x)) # needed to drop duplicates
table_schemata = table_schemata[['header', 'fits']].drop_duplicates().reset_index(drop=True)
table_schemata['header'] = table_schemata['header'].apply(lambda x: x.split('<|>')) # rebuild original state
table_schemata.head()

,header,fits
0,"[Date, Time, ACC Team, Big Ten Team, Location,...",
1,"[Institution, Wins, Loss, Home Wins, Home Loss...",
2,"[Pick #, MLS Team, Player, Position, Affiliation]",
3,"[DVD title, Number of Episodes, Region 2, Regi...",
4,"[Year, Coach, Crew, Record, Win %]",


In [4]:
table_schemata = table_schemata.head(10000)

In [5]:
all_headers = []
_ = [all_headers.extend(header) for header in table_schemata['header'].values]
all_headers = list(set(all_headers))
random.shuffle(all_headers)
candidates = [column \
              .replace('/', ' ') \
              .replace('_', ' ') \
              for column in all_headers[:1000]]
cache = {candidate: np.mean([w2v(word.lower()) for word in candidate.split()], axis=0) for candidate in candidates}
if cache.get(''):
    del cache['']

In [6]:
from collections import defaultdict

def calculate_fits(header, cache):
    eps = 0.001
    embs = [np.mean([w2v(word.lower()) for word in col.split()], axis=0) for col in header]
    fits = defaultdict(list)
    for candidate, embedding in cache.items():
        try:
            vectors = [embedding] + embs
            similarity = cosine_similarity(vectors)[0][1:]
            max_sim = np.max(similarity)
            if abs(max_sim + eps) >= 1.0:
                continue
            max_pos = np.argmax(similarity)

            # append the best candidate with its similarity to the header
            # to the lists of candidates for the given header
            # i.e. fits['ACC Team'] -> [('Team', 0.75), ('Coach', 0.64), (<new candidate>, <similarity>)]
            fits[header[max_pos]].append((candidate, max_sim))
        except:
            continue
            #print("HEADER: ", header, "\nCANDIDATE:", candidate)
    return fits

def reduce_fits(fits, lower_threshold=0.9):
    best_fits = {}
    for column, candidate_tuples in fits.items():
        best_candidate = sorted(candidate_tuples, key=lambda x: x[1], reverse=True)[0]
        similarity = best_candidate[1]
        if similarity > lower_threshold:
            best_fits[column] = best_candidate[0]
    return best_fits

for idx, row in tqdm(table_schemata.iterrows(), total=len(table_schemata)):
    header = row['header']
    fits = calculate_fits(header, cache)
    best_fits = reduce_fits(fits)
    row['fits'] = best_fits
    table_schemata.loc[idx] = row

  2%|█▍                                                                             | 180/9899 [00:29<23:21,  6.94it/s]C:\Users\johan\AppData\Local\Continuum\anaconda3\envs\pointer_sort\lib\site-packages\numpy\core\fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\johan\AppData\Local\Continuum\anaconda3\envs\pointer_sort\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
  2%|█▍                                                                             | 183/9899 [00:29<21:10,  7.65it/s]C:\Users\johan\AppData\Local\Continuum\anaconda3\envs\pointer_sort\lib\site-packages\numpy\core\fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\johan\AppData\Local\Continuum\anaconda3\envs\pointer_sort\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / 

 18%|██████████████▏                                                               | 1804/9899 [05:16<21:17,  6.34it/s]C:\Users\johan\AppData\Local\Continuum\anaconda3\envs\pointer_sort\lib\site-packages\numpy\core\fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\johan\AppData\Local\Continuum\anaconda3\envs\pointer_sort\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
 19%|██████████████▉                                                               | 1892/9899 [05:31<24:14,  5.50it/s]C:\Users\johan\AppData\Local\Continuum\anaconda3\envs\pointer_sort\lib\site-packages\numpy\core\fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\johan\AppData\Local\Continuum\anaconda3\envs\pointer_sort\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / 

 43%|█████████████████████████████████▏                                            | 4217/9899 [12:18<17:24,  5.44it/s]C:\Users\johan\AppData\Local\Continuum\anaconda3\envs\pointer_sort\lib\site-packages\numpy\core\fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\johan\AppData\Local\Continuum\anaconda3\envs\pointer_sort\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
 44%|██████████████████████████████████▋                                           | 4403/9899 [12:47<13:51,  6.61it/s]C:\Users\johan\AppData\Local\Continuum\anaconda3\envs\pointer_sort\lib\site-packages\numpy\core\fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\johan\AppData\Local\Continuum\anaconda3\envs\pointer_sort\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / 

 61%|███████████████████████████████████████████████▍                              | 6016/9899 [17:31<10:03,  6.44it/s]C:\Users\johan\AppData\Local\Continuum\anaconda3\envs\pointer_sort\lib\site-packages\numpy\core\fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\johan\AppData\Local\Continuum\anaconda3\envs\pointer_sort\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
 61%|███████████████████████████████████████████████▍                              | 6017/9899 [17:31<10:29,  6.17it/s]C:\Users\johan\AppData\Local\Continuum\anaconda3\envs\pointer_sort\lib\site-packages\numpy\core\fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\johan\AppData\Local\Continuum\anaconda3\envs\pointer_sort\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / 

 81%|███████████████████████████████████████████████████████████████▏              | 8024/9899 [23:24<05:44,  5.44it/s]C:\Users\johan\AppData\Local\Continuum\anaconda3\envs\pointer_sort\lib\site-packages\numpy\core\fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\johan\AppData\Local\Continuum\anaconda3\envs\pointer_sort\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
 83%|████████████████████████████████████████████████████████████████▍             | 8180/9899 [23:51<05:27,  5.24it/s]C:\Users\johan\AppData\Local\Continuum\anaconda3\envs\pointer_sort\lib\site-packages\numpy\core\fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\johan\AppData\Local\Continuum\anaconda3\envs\pointer_sort\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / 

100%|██████████████████████████████████████████████████████████████████████████████| 9899/9899 [28:57<00:00,  5.70it/s]


In [7]:
table_schemata.head()

,header,fits
0,"[Date, Time, ACC Team, Big Ten Team, Location,...",{'Time': 'Start Time'}
1,"[Institution, Wins, Loss, Home Wins, Home Loss...",{}
2,"[Pick #, MLS Team, Player, Position, Affiliation]",{}
3,"[DVD title, Number of Episodes, Region 2, Regi...","{'Number of Episodes': 'Number of episode', 'R..."
4,"[Year, Coach, Crew, Record, Win %]",{'Year': 'Year ended'}


In [8]:
# Drop rows that have not been tested
table_schemata.drop(table_schemata.loc[table_schemata['fits'] == ''].index, inplace=True)

In [15]:
rows = []
for idx, ts_row in tqdm(table_schemata.iterrows()):
    header, fits = ts_row[['header', 'fits']]
    num_words_in_header = len((' '.join(header)).split(' '))
    if num_words_in_header > 30:
        continue
    for target_col, source_col in fits.items(): 
        if 'k {\math' in source_col:
            continue
        seq_row = {
            'source_col': source_col,
            'input_cols': '<|>'.join(header),
            'target_col': target_col
        }
        rows.append(seq_row)
rows = pd.DataFrame(rows)
rows.to_csv('data/training/schema_matching_raw.txt', index=False)

9899it [00:04, 2058.82it/s]


In [16]:
rows.shape

(5889, 3)

Vorgehen:
1. für jede Tabelle eine Spalte finden, die aus Quellschema sein könnte -> die als Eingabe  
Format: Quellspalte -> [Zielspalte]